## Finance Notebook
Mark Prettyman 
All Rights Reserved.

# NOTE:
- Restart kernel HERE before moving on

In [4]:
import pandas as pd
import yfinance as yf
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


In [19]:
import requests
import pandas as pd

# API endpoint to retrieve the available tickers
url = "https://rapidapi.com/yfinance/api/yfinance"

# Make a GET request to the API
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Convert the API response to a pandas DataFrame
    tickers = pd.DataFrame(response.json())
    
    # Get the list of available tickers
    tickers_list = tickers['symbol'].tolist()
    
    # Define the list of stocks to generate a signal for
    stocks = tickers_list[:5]
    
    # Generate the signal for each stock in the list
    for stock in stocks:
        # Code to generate the signal for the stock
        signal = None # replace with the code to generate the signal
        
        # Print the signal for the stock
        print(f"Signal for stock {stock}: {signal}")
else:
    # Print an error message if the request was not successful
    print(f"Failed to retrieve the available tickers. Response code: {response.status_code}")

Failed to retrieve the available tickers. Response code: 404


# Defining BSM
- Black Scholes Merton Formula to determine true option pricing


In [8]:
>>> S=100
>>> K=95
>>> q=.05
>>> t = 0.5
>>> r = 0.1
>>> sigma = 0.2

In [12]:
# Create a function to generate moving averages
def create_moving_average(data, window, type='sma'):
    if type == 'sma':
        data['moving_average'] = data['close'].rolling(window=window).mean()
    elif type == 'ema':
        data['moving_average'] = data['close'].ewm(span=window).mean()
    else:
        raise ValueError("Invalid moving average type")
    return data

# Create a function to generate buy/sell signals
def generate_signals(data, moving_average_type='sma', window=10):
    data = create_moving_average(data, window, type=moving_average_type)
    buy_signals = []
    sell_signals = []
    for i in range(len(data)):
        if i == 0:
            continue
        if data['close'][i] > data['moving_average'][i] and data['close'][i-1] <= data['moving_average'][i-1]:
            buy_signals.append(data['date'][i])
        if data['close'][i] < data['moving_average'][i] and data['close'][i-1] >= data['moving_average'][i-1]:
            sell_signals.append(data['date'][i])
    return buy_signals, sell_signals


In [14]:
tickers = yf.Tickers("^GSPC ^DJI ^IXIC")
df = tickers.history(period="max")

[*********************100%***********************]  3 of 3 completed


In [17]:
df.head()

Close                  Dividends             High  \
                           ^DJI      ^GSPC ^IXIC      ^DJI ^GSPC ^IXIC ^DJI   
Date                                                                          
1927-12-30 00:00:00-05:00   NaN  17.660000   NaN       NaN   0.0   NaN  NaN   
1928-01-03 00:00:00-05:00   NaN  17.760000   NaN       NaN   0.0   NaN  NaN   
1928-01-04 00:00:00-05:00   NaN  17.719999   NaN       NaN   0.0   NaN  NaN   
1928-01-05 00:00:00-05:00   NaN  17.549999   NaN       NaN   0.0   NaN  NaN   
1928-01-06 00:00:00-05:00   NaN  17.660000   NaN       NaN   0.0   NaN  NaN   

                                            Low  ...       Open             \
                               ^GSPC ^IXIC ^DJI  ... ^IXIC ^DJI      ^GSPC   
Date                                             ...                         
1927-12-30 00:00:00-05:00  17.660000   NaN  NaN  ...   NaN  NaN  17.660000   
1928-01-03 00:00:00-05:00  17.760000   NaN  NaN  ...   NaN  NaN  17.760000   
1928-01-04 00:00:00-05:00  17.719999   NaN  NaN  ...   NaN  NaN  17.719999   
1928-01-05 00:00:00-05:00  17.549999   NaN  NaN  ...   NaN  NaN  17.549999   
1928-01-06 00:00:00-05:00  17.660000   NaN  NaN  ...   NaN  NaN  17.660000   

                                Stock Splits             Volume              
                          ^IXIC         ^DJI ^GSPC ^IXIC   ^DJI ^GSPC ^IXIC  
Date                                                                         
1927-12-30 00:00:00-05:00   NaN          NaN   0.0   NaN    NaN     0   NaN  
1928-01-03 00:00:00-05:00   NaN          NaN   0.0   NaN    NaN     0   NaN  
1928-01-04 00:00:00-05:00   NaN          NaN   0.0   NaN    NaN     0   NaN  
1928-01-05 00:00:00-05:00   NaN          NaN   0.0   NaN    NaN     0   NaN  
1928-01-06 00:00:00-05:00   NaN          NaN   0.0   NaN    NaN     0   NaN  

[5 rows x 21 columns]

In [16]:
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Define the stocks you want to track
stocks = []

# Define the moving average type and window
moving_average_type = 'sma'
window = 10

# Define the machine learning model
model = LinearRegression()

# Define the expiration date for the options
expiration_date = pd.to_datetime('2022-12-31')

while True:
    # Download the latest market data for the stocks
    stock_data = yf.download(stocks, start='2022-01-01', end='2022-12-31')
    
    # Iterate through each stock
    for stock in stocks:
        # Generate buy/sell signals for the current stock
        if moving_average_type == 'sma':
            stock_data[stock]['sma'] = stock_data[stock]['Close'].rolling(window=window).mean()
            buy_signals = np.where(stock_data[stock]['Close'] > stock_data[stock]['sma'], 1, 0)
            sell_signals = np.where(stock_data[stock]['Close'] < stock_data[stock]['sma'], 1, 0)
        elif moving_average_type == 'ema':
            stock_data[stock]['ema'] = stock_data[stock]['Close'].ewm(span=window).mean()
            buy_signals = np.where(stock_data[stock]['Close'] > stock_data[stock]['ema'], 1, 0)
            sell_signals = np.where(stock_data[stock]['Close'] < stock_data[stock]['ema'], 1, 0)
        
        # Prepare the data for the machine learning model
        X = stock_data[stock][['Close']][:-1]
        y = stock_data[stock]['Close'].shift(-1)[:-1]
        
        # Fit the model
        model.fit(X, y)
        
        # Make predictions
        predictions = model.predict(X)
        
        # Calculate the mean squared error
        mse = mean_squared_error(y, predictions)
        
        # Print the results
        print(f"{stock} Buy signals: ", buy_signals)
        print(f"{stock} Sell signals: ", sell_signals)
        print(f"{stock} Mean Squared Error: {mse}")
        
        # Get the option data for the stock
        option_data = yf.option_chain(stock).calls
        
        # Get the last closing price for the stock
        spot_price = stock


           


[*********************100%***********************]  1 of 1 completed


KeyError: 'GOOGL'

In [ ]:
# Create a function to generate moving averages
def create_moving_average(data, window, type='sma'):
    if type == 'sma':
        data['moving_average'] = data['close'].rolling(window=window).mean()
    elif type == 'ema':
        data['moving_average'] = data['close'].ewm(span=window).mean()
    else:
        raise ValueError("Invalid moving average type")
    return data

# Create a function to generate buy/sell signals
def generate_signals(data, moving_average_type='sma', window=10):
    data = create_moving_average(data, window, type=moving_average_type)
    buy_signals = []
    sell_signals = []
    for i in range(len(data)):
        if i == 0:
            continue
        if data['close'][i] > data['moving_average'][i] and data['close'][i-1] <= data['moving_average'][i-1]:
            buy_signals.append(data['date'][i])
        if data['close'][i] < data['moving_average'][i] and data['close'][i-1] >= data['moving_average'][i-1]:
            sell_signals.append(data['date'][i])
    return buy_signals, sell_signals

# Generate buy/sell signals using a SMA
buy_signals_sma, sell_signals_sma = generate_signals(df, moving_average_type='sma', window=10)

# Generate buy/sell signals using a EMA
buy_signals_ema, sell_signals_ema = generate_signals(df, moving_average_type='ema', window=10)

# Print the signals
print("Buy signals (SMA): ", buy_signals_sma)
print("Sell signals (SMA): ", sell_signals_sma)
print("Buy signals (EMA): ", buy_signals_ema)
print("Sell signals (EMA): ", sell_signals_ema)